<a href="https://colab.research.google.com/github/danielf-moralesp20/neural-network-projects/blob/main/handwritten-digit-recognizer/handwritten-digit-recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import tensorflow_datasets as tfds

In [59]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'mnist',
    split=['train[:80%]+test[:80%]', 'train[80%:90%]+test[80%:90%]', 'train[90%:]+test[90%:]'],
    shuffle_files=True,
    with_info=True,
    as_supervised=True,
)